In [53]:
#import libraries
import pandas as pd
import random
from collections import defaultdict, deque
import heapq
import math
import osmnx as ox
import pandas as pd
import os
import bisect
from math import radians, sin, cos, sqrt, atan2
from shapely.wkt import loads
from geopy.distance import geodesic

In [22]:
# create the graph node
class Node():
    def __init__(self, id, street_count, coordinates):
        self.coordinates = coordinates
        self.street_count = street_count
        self.id = id
        self.neighbors = set()

        # information about the type of node (charging or not)
        self.charge = True

    def __lt__(self, other):
        # Define how nodes should be compared (e.g., based on their names)
        return self.name < other.name

In [107]:
# create the graph
class Graph():
    def __init__(self):
        self.nodes = set()
        self.coordinates_list = [(float('-inf'), float('-inf')), (float('inf'), float('inf'))]
        self.get_node = {} # getting node from node_id
        self.get_node_from_coordinates = {}
        self.distance = defaultdict(dict)

    def add_node(self, id, street_count, coordinates):
        node = Node(id, street_count, coordinates)
        self.nodes.add(node)
        self.get_node[id] = node
        self.coordinates_list.insert(bisect.bisect_left(self.coordinates_list, coordinates), coordinates)
        self.get_node_from_coordinates[coordinates] = node
    
    def __getitem__(self, node_id):
        return self.get_node[node_id]
    
    def add_edge(self, node1, node2, distance, oneway=False):
        node1.neighbors.add(node2)
        node2.neighbors.add(node1)
        self.distance[node1][node2] = distance
        if not oneway:
            self.distance[node2][node1] = distance

    def nearest_node(self, coordinates):
        index = bisect.bisect_left(self.coordinates_list, coordinates)
        if self.coordinates_list[index] == coordinates:
            return coordinates
        left_distance = geodesic(self.coordinates_list[index - 1], coordinates)
        right_distance = geodesic(self.coordinates_list[index], coordinates)
        if left_distance < right_distance:
            return self.coordinates_list[index - 1]
        else:
            return self.coordinates_list[index]
    
    def haversine_distance(self, coordinates1, coordinates2):
        # coordinates -> [latitude, longitude]
        # Convert latitude and longitude from degrees to radians
        lat1, lon1, lat2, lon2 = map(radians, [coordinates1[0], coordinates1[1], coordinates2[0], coordinates2[1]])

        # Haversine formula
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        # Radius of the Earth in meters (approximate)
        R = 6371000

        # Calculate the distance
        distance = R * c
        return distance

    # find the shortest path between two nodes
    def shortest_path(self, source, destination):
        # get the start and end nodes
        start_coordinates, end_coordinates = self.nearest_node(source), self.nearest_node(destination)
        start_node, end_node = self.get_node_from_coordinates[start_coordinates], self.get_node_from_coordinates[end_coordinates]

        # Function to calculate actual geographical distance between a node and the end node
        #print(len([coordinates, end_coordinates]))
        distance_from_end = lambda coordinates:  geodesic(coordinates, end_coordinates).meters

        # coefficients for the heuristic function
        h1 = 1 # coefficient for the heuristic distance from end point
        h2 = 1 # coefficient for the distance
        # h3 = 1 # coefficient for the time -> for later use
        full_charge = float('inf')  # charge in terms of distance


        # find the shortest path
        visited = {}
        queue = [[0, 0, full_charge, start_node, [start_node.id]]] # [heuristic value, SoC(in distance), distance, node, path]
        count = 1
        while queue:
            count += 1
            heuristic, distance, charge, node, path = heapq.heappop(queue)
            if node == end_node:
                return distance, charge, path, len(queue)
            if node.charge: # fully charge at charging node
                charge = full_charge
            if node not in visited or visited[node] > heuristic:
                visited[node] = heuristic
                for neighbor in node.neighbors:
                    new_charge = charge - self.distance[node][neighbor]
                    #if new_charge < 10:
                       # continue
                    neighbor_heuristic = h1 * (distance + self.distance[node][neighbor]) + h2 * distance_from_end(neighbor.coordinates)
                    heapq.heappush(queue, [neighbor_heuristic, distance + self.distance[node][neighbor], new_charge, neighbor, path + [neighbor.id]])
        print(count)
        return float('inf'), 0, [], 0
graph = Graph()


In [97]:
# Define the file paths for nodes.csv, edges.csv, and charging.csv within the 'data' folder
nodes_file = os.path.join('', 'nodes.csv')
edges_file = os.path.join('', 'edges.csv')
charging_file = os.path.join('', 'charging.csv')

# Load nodes.csv, edges.csv, and charging.csv into pandas DataFrames
nodes_df = pd.read_csv(nodes_file)
edges_df = pd.read_csv(edges_file)
charging_df = pd.read_csv(charging_file)

C:\Users\91735\AppData\Local\Temp\ipykernel_17164\1941451539.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  nodes_df = pd.read_csv(nodes_file)
C:\Users\91735\AppData\Local\Temp\ipykernel_17164\1941451539.py:8: DtypeWarning: Columns (8,10,11,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(edges_file)


In [108]:
for idx, row in nodes_df.iterrows():
    graph.add_node(row['node_id'], row['street_count'], (row['longitude'], row['latitude']))

In [109]:

# Function to extract coordinates from LINESTRING
def extract_coordinates(line_string):
    line = loads(line_string)
    return list(line.coords)

# Apply the function to the 'geometry' column and save coordinates into a new column 'coordinates'
coordinates_lists = edges_df['geometry'].apply(extract_coordinates)


In [110]:
mean = 0
for i, coordinate_list in enumerate(coordinates_lists):
    distance = 0
    for i in range(len(coordinate_list) - 1):
        coordinates1, coordinates2 = graph.nearest_node(coordinate_list[i]), graph.nearest_node(coordinate_list[i + 1])
        distance += geodesic(coordinates1, coordinates2).meters
        graph.add_edge(graph.get_node_from_coordinates[coordinates1], graph.get_node_from_coordinates[coordinates2], distance)

In [102]:
total, absent = 0, []
for idx, row in edges_df.iterrows():
    if idx == 10: break
    coordinates = coordinates_lists[idx]
    for i in range(len(coordinates) - 1):
        distance = geodesic(coordinates[i], coordinates[i-1]).meters
        if coordinates[i] not in graph.get_node_from_coordinates:
            absent.append(coordinates[i])
        total += 1
print(total, absent)
        #graph.add_edge(graph.get_node_from_coordinates[coordinates[i]], graph.get_node_from_coordinates[coordinates[i + 1]], distance, row['oneway'])

33 [(77.1701288, 28.5601606), (77.1696501, 28.5606231), (77.1693342, 28.5609282), (77.1689618, 28.5613716), (77.1689341, 28.5614046), (77.1750377, 28.5805671), (77.1751091, 28.5804897), (77.175349, 28.5800666), (77.1754589, 28.5798727), (77.1750168, 28.5804549), (77.177488, 28.5762264), (77.1925453, 28.5691535), (77.1925276, 28.5691531), (77.1919236, 28.5691411), (77.1910585, 28.5692287), (77.1919093, 28.5692373), (77.1910877, 28.5693006), (77.1875752, 28.5696885), (77.1674837, 28.5950505), (77.1682118, 28.5954343), (77.1686555, 28.5956681), (77.1690725, 28.5958879), (77.1734944, 28.5982286)]


In [266]:
for idx, row in nodes_df.iterrows():
    graph.add_node(int(row['node_id']), int(row['street_count']), tuple([row['latitude'], row['longitude']]))

In [267]:
for id in charging_df['node_id']:
    graph.get_node[id].charge = True

In [268]:
for idx, row in edges_df.iterrows():
    if row['length'] != "None":
        graph.add_edge(graph.get_node[row['node1']], graph.get_node[row['node2']], float(row['length']))

In [105]:
node1 = random.choice(list(graph.nodes))
node2 = random.choice(list(graph.nodes))

In [111]:
print(node1.id, node2.id)
graph.shortest_path(node1.coordinates, node2.coordinates)

144422 117910


(12231.551827180134,
 inf,
 [144422,
  165029,
  165025,
  165024,
  165021,
  165020,
  5609,
  5626,
  6044,
  5645,
  6040,
  164988,
  5844,
  5630,
  6000,
  5674,
  5905,
  165008,
  5617,
  5686,
  6012,
  5740,
  5735,
  5728,
  165012,
  5732,
  5620,
  5592,
  5706,
  5762,
  5677,
  164974,
  164975,
  164924,
  6006,
  6041,
  5869,
  5835,
  6017,
  5938,
  5881,
  5860,
  5804,
  5894,
  5865,
  5988,
  5823,
  6038,
  5854,
  5782,
  5875,
  5811,
  104483,
  92437,
  95270,
  95267,
  144167,
  95276,
  95272,
  95285,
  95286,
  94432,
  95287,
  94434,
  98642,
  94395,
  98641,
  98637,
  95544,
  174380,
  94437,
  94463,
  175234,
  175211,
  175212,
  98659,
  94467,
  94461,
  95535,
  95534,
  95532,
  95521,
  95520,
  143730,
  95526,
  37979,
  105485,
  109360,
  167668,
  167666,
  167664,
  167660,
  167662,
  109361,
  167605,
  109362,
  167589,
  109363,
  109364,
  95357,
  144387,
  33582,
  110962,
  33581,
  141433,
  176033,
  116158,
  105110,
  1

In [234]:
counter = defaultdict(int)
for node in list(graph.nodes):
    counter[len(node.neighbors)] += 1
print(counter)

defaultdict(<class 'int'>, {2: 46197, 1: 48353, 0: 22585, 3: 53066, 4: 12019, 5: 7})


In [114]:
path = [144422, 165029, 165025, 165024, 165021, 165020, 5609, 5626, 6044, 5645, 6040, 164988, 5844, 5630, 6000, 5674, 5905, 165008, 5617, 5686, 6012, 5740, 5735, 5728, 165012, 5732, 5620, 5592, 5706, 5762, 5677, 164974, 164975, 164924, 6006, 6041, 5869, 5835, 6017, 5938, 5881, 5860, 5804, 5894, 5865, 5988, 5823, 6038, 5854, 5782, 5875, 5811, 104483, 92437, 95270, 95267, 144167, 95276, 95272, 95285, 95286, 94432, 95287, 94434, 98642, 94395, 98641, 98637, 95544, 174380, 94437, 94463, 175234, 175211, 175212, 98659, 94467, 94461, 95535, 95534, 95532, 95521, 95520, 143730, 95526, 37979, 105485, 109360, 167668, 167666, 167664, 167660, 167662, 109361, 167605, 109362, 167589, 109363, 109364, 95357, 144387, 33582, 110962, 33581, 141433, 176033, 116158, 105110, 105111, 143857, 105112, 105108, 168800, 168798, 168796, 168794, 168792, 168790, 168788, 35677, 168784, 137854, 137903, 137853, 168836, 168831, 137910, 137916, 137915, 137913, 137856, 168811, 168778, 168780, 32775, 34706, 115020, 32776, 32762, 32758, 4976, 4975, 4994, 5010, 116142, 98120, 90104, 165144, 165142, 165055, 165053, 165045, 165044, 165042, 165036, 165041, 165040, 165032, 90135, 90137, 90134, 90138, 90136, 90133, 90130, 98073, 105270, 105317, 105301, 105306, 105315, 105309, 105308, 105307, 172249, 105266, 105265, 105287, 105286, 165121, 114781, 114778, 114779, 103212, 165107, 103211, 114589, 114784, 114599, 114588, 114587, 114756, 114591, 114600, 114601, 114603, 98065, 165215, 165217, 165218, 98066, 108631, 108632, 34203, 140390, 140425, 110954, 117437, 102055, 114713, 114714, 141167, 114715, 99998, 99996, 40121, 106940, 40125, 164219, 119845, 164246, 173792, 117885, 106895, 107304, 106894, 107308, 106892, 31975, 117879, 31974, 138106, 117875, 117876, 138377, 97856, 138110, 97880, 97853, 97900, 108447, 138582, 108443, 106813, 138294, 40092, 169664, 106316, 40091, 92480, 92479, 92483, 92487, 79128, 103736, 103738, 103739, 79125, 91680, 140096, 117907, 117910]

In [115]:
nodes = [graph.get_node[id] for id in path]

In [116]:
coordinates = [node.coordinates for node in nodes]

In [117]:
print(coordinates)

[(77.1771472, 28.6808446), (77.1773266, 28.6807528), (77.1774913, 28.6809791), (77.1769106, 28.6812818), (77.1767026, 28.6813818), (77.1767659, 28.6814862), (77.1763313, 28.6816999), (77.1760819, 28.6818226), (77.1759052, 28.6819328), (77.1758506, 28.6819668), (77.1757308, 28.6820415), (77.175598, 28.6821242), (77.1755624, 28.6821464), (77.1754219, 28.682234), (77.1753864, 28.6822639), (77.1753152, 28.6823006), (77.1751709, 28.6823905), (77.175145, 28.6824066), (77.1749804, 28.6825093), (77.1748523, 28.6825891), (77.1747777, 28.6826356), (77.1744865, 28.6828171), (77.1743684, 28.6828908), (77.1738869, 28.6823629), (77.1737207, 28.6824791), (77.1735576, 28.6825931), (77.1734224, 28.6826876), (77.1729192, 28.6821578), (77.1727447, 28.6822781), (77.1725542, 28.6824095), (77.1721058, 28.6819547), (77.1719474, 28.6820542), (77.1713061, 28.6814143), (77.1711748, 28.6800256), (77.170667, 28.6798416), (77.1698949, 28.6795622), (77.1697243, 28.6795005), (77.1697, 28.6794917), (77.1695208, 28.67